Running a fresh DART-ROMS experiment on RENCI requires configuring ROMS and configuring DART. Below is a CNAPS example.

**1) Clone the repo**

You will have to be a collaborator on the git project. Use your NCSU unity login credentials.

**2) Go inside the ROMS directory**

We are currently using a ROMS version from May 2018.

**3) Set the MY_ROOT_DIR path in build_Mao_renci.bash to your new ROMS directory**

this path

**4) Load necessary NETCDF/etc. libraries**

**5) Build ROMS for EAKF**

The EAKF configuration is set in the useast.h file

 This creates the ROMS executable “oceanM”, which will be used to launch all instances of ROMS for the ensemble. If you want to change an option in ROMS you can recompile without re-staging the entire ensemble, but you must cp oceanM to the path of your working ensemble. 

**6) Locate and aggregate necessary files in a central directory. Name it something like “ncfiles”**

cnaps_ocean.in.template could point directly to forcing/boundary/superobs/ini files in Shun’s directory, but SLURM seems to have trouble reaching over to another user’s directory for files during the simulation so might have copied these to our own work space in /scratch/rhe/cnaps/ncfiles directory.

	For ROMS:
		forcing
		boundary
		grid

      For EAKF:
        sampling error correction table with desired ensemble size
        perturbation files for each member
        super obs
This has to be in a strange directory —> /ENKF/rma/assimilation_code/programs/system_simulation/work

Might as well double check this

There’s 31 at the end there. See here about how to add other ensemble sizes (https://www.image.ucar.edu/DAReS/DART/Manhattan/assimilation_code/programs/gen_sampling_err_table/gen_sampling_err_table.html)

Perturbation files were created on Cheyenne where Haibo’s 30yr hindcast is located
     super obs

Super obs are in Shun's directory

CNAPS perturbation files are in: /scratch/rhe/cnaps/ncfiles/april_1

**7) Time to set up stuff in the Ensemble directory. Check settings in ocean.in.template**

Here’s a diff between CNAPS and GoM ocean.in.template files. Notice the difference in grid points, etc. Anywhere there is a path you need to change it to your own. 

Path to varinfo.dat should be your own

Handling of boundary conditions. Change RadNud to Clamped for now, though, for consistency with Shun’s 4DVAR configuration. Notice how the GOM domain has North and South boundaries, unlike CNAPS.

Timestep and related timing variables. Double the tilmestep means double the NTIMES for a 2 day run. FGAT

Grid file path


Boundary condition file path


Forcing file path


Decide how many processors you want each member to use. In this case, 64:


**8) Copy or link initial conditions for ensemble from ncfiles to ROMS_MAY2018/Ensemble**


**9) Change super obs path in s4dvar.in.template**

Make that file exist


---------------

# 11) Prepare DART

There are several scripts you need to edit before "staging" the experiment.


**11a)   vi advance_ensemble.csh.template**

This is the script that will make the ensemble members run over and over again (cycle). Important things here are
 - how many members do you want to run at a time?

  - where do you want the job to run?

  - where is the ROMS directory?

**11b) vi stage_experiment_renci.csh**

This script creates a directory where the experiment will take place and copies everything you need to it. 

- where are the important directories directory?

Fix these

Things like this won’t matter unless you soft code them in ocean.in.template. I hard code them for some reason. The only ones that **need** to be soft coded are DSTART and ININAME because they’ll be updated after every filter cycle.

By soft code I mean putting the actual number in the ocean.in.template instead of putting "MyNtileI" or whatever, which would be replaced by the numbers below when this script runs. If you don't put "MyXXXXXX" in ocean.in.template then these calls to $SUBSTITUTE won't change anything. 

**11c) vi submit_multiple_jobs_slurm.csh**

This is the script you wil use to start and restart the ensemble. 
- How many cycles do you want to run?

11d) vi run_filter.csh.template

This is the script that controls the filter, which runs once all the ROMS instances have finished.
- How many cores would you like the filter to use?
- Where are we running?

**11e) Check DART configuration.**

This is the namelist that controls actual DART behavior. The previous stuff just controls manages how files are copied, updated, etc, from cycle to cycle. 

**12) STAGE THE EXPERIMENT**


**13) Navigate to the experiment directory and launch it**


Launch experiment


**14) Check on experiment.**


See all the filter instances waiting? That’s how many cycles are left. 

Final step: notice that things have gone horribly wrong. 

It’s crashing. Cancel the job

Now determine why it’s crashing. Was it during the ROMS part of the job or the DART (filter) part of the job? ROMS:

Fix this and Re-stage experiment. 


Success!